### Evaluation Code

This code relates to the paper "Measuring Financial Time Series Similarity With a View to Identifying Profitable Stock Market Opportunities" which was published in the proceedings of the International Conference on Case Based Reasoning (ICCBR) 2021

For queries please email rian.dolphin@ucdconnect.ie

In [ ]:
#-- Imports
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
#from scipy.special import softmax
import time

import datetime as dt
import random
import copy

from scipy import stats

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

#-- Uncomment these for white Theme
my_template = 'plotly_white'
background_color = 'rgba(255,255,255,1)'

#-- Dark Theme
#my_template = 'plotly_dark'
#background_color = 'rgba(0,0,0,1)'

#-- Dark Transparent background
#my_template = 'plotly_dark'
#background_color = 'rgba(0,0,0,0)'

In [2]:
#-- Load in from saved files
import os, os.path

def sort_date(df):
    df.start_date = pd.to_datetime(df.start_date, unit='ms')
    df.end_date = pd.to_datetime(df.end_date, unit='ms')
    df.sort_values(by=['start_date'], inplace=True)
    return df

# simple version for working with CWD
#print(len([name for name in os.listdir('All_Windows/')]))

rolling_dfs = []
path = 'Windows_2021_Short/'
files = [name for name in os.listdir(path)]
files = [file for file in files if file.startswith('window')]
count=1
for file_name in files:
    temp_df = pd.read_json(path+'window_'+str(count)+'.json')
    temp_df = sort_date(temp_df)
    rolling_dfs.append(temp_df)
    #df.to_json(file_name)
    count+=1
    if count==250:
        break
print('=== Rolling Dfs Loaded ===')

=== Rolling Dfs Loaded ===


In [3]:
def sort_by_metric(rolling_dfs, metric, reverse=True):
    """
    Function which will order the cases by the given similarity metric
    """
    for df_i in tqdm(range(len(rolling_dfs))):
        df = rolling_dfs[df_i].copy()
        df = df.astype('object')
        for i in range(len(df)):
            temp = df.iloc[i][7:]
            df.loc[df.index[i], metric], df.loc[df.index[i], 'sim_idx_val'], df.loc[df.index[i], 'sim_next_month'], df.loc[df.index[i], 'sim_next_year'], df.loc[df.index[i], 'correlations'], df.loc[df.index[i], 'cumprod_diffs'], df.loc[df.index[i], 'shape_measure'], df.loc[df.index[i], 'equal_weight_similarity'], df.loc[df.index[i], 'quantdare']=zip(*sorted(zip(temp[metric], temp['sim_idx_val'], temp['sim_next_month'], temp['sim_next_year'],temp['correlations'], temp['cumprod_diffs'], temp['shape_measure'], temp['equal_weight_similarity'], temp['quantdare']), reverse=reverse))
        rolling_dfs[df_i] = df
    return copy.deepcopy(rolling_dfs)

### Get the results seen in Table 1

In [4]:
def get_error(rolling_dfs,k, aggregation=False, seed=False):
    if seed!=False:
        np.random.seed(seed)
        tickers = list(rolling_dfs[0].ticker.value_counts().index)
        tickers = np.random.choice(tickers, int(len(tickers)*0.2), replace=False)
        
        rolling_dfs = [df[~df.ticker.isin(tickers)] for df in rolling_dfs]
        
        x_top = np.array([abs((np.array([np.array(xi)[:k] for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values])).flatten()) for df in rolling_dfs]).flatten()
        
        
    elif aggregation:
        #x_top = np.array([abs((np.array([np.array(xi)[:k] for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values]).mean()-df[df.start_date==df.start_date.iloc[0]].next_month.values.reshape(-1,1)).flatten()) for df in rolling_dfs]).flatten()
        x_top = np.array([(np.array([np.random.choice(np.array(xi),k,replace=False) for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values]).mean()).flatten() for df in rolling_dfs]).flatten()
        #x_top = np.array([(np.array([np.array(xi)[:k] for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values]).mean()).flatten() for df in rolling_dfs]).flatten()
    else:
        #-- Use line below for errors (Table 1)
        x_top = np.array([abs((np.array([np.array(xi)[:k] for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values])-df[df.start_date==df.start_date.iloc[0]].next_month.values.reshape(-1,1)).flatten()) for df in rolling_dfs]).flatten()
        #-- Use line below for random!
        #x_top = np.array([abs((np.array([np.random.choice(np.array(xi),k,replace=False) for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values])-df[df.start_date==df.start_date.iloc[0]].next_month.values.reshape(-1,1)).flatten()) for df in rolling_dfs]).flatten()
        #-- Use line below for trading sim? NO SINCE ABS
        #x_top = np.array([abs((np.array([np.array(xi)[:k] for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values])).flatten()) for df in rolling_dfs]).flatten()
        #-- line below for Random Trading sim
        #x_top = np.array([abs((np.array([np.random.choice(np.array(xi),k,replace=False) for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values])).flatten()) for df in rolling_dfs]).flatten()
    #x_bottom = np.array([abs((np.array([np.array(xi)[-k:] for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values])-df[df.start_date==df.start_date.iloc[0]].next_month.values.reshape(-1,1)).flatten()) for df in rolling_dfs]).flatten()
    #x_rand = np.array([abs((np.array([np.array(xi)[[np.random.randint(0,len(xi)) for i in range(k)]] for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values])-df[df.start_date==df.start_date.iloc[0]].next_month.values.reshape(-1,1)).flatten()) for df in rolling_dfs]).flatten()
    #print(x_top.shape)
    return x_top.mean(), x_top.std(), x_top


def add_to_error_dict(rolling_dfs, metric, aggregation=False, seed=False):
    temp_dict = {}
    for k in [1,2,5,10,25,50]:
        temp_dict[k] = get_error(rolling_dfs,k, aggregation=aggregation, seed=seed)
    error_dict[metric] = temp_dict
    


In [5]:
#-- Define empty dictionary to store errors
error_dict = {}
#-- Set aggregation parameter
agg = False

#-- Add the proposed metric
print("ADDING ProposedAdjusted")
for alpha in [0.5]:
    print('='*20)
    beta=1-alpha
    print(f'alpha = {alpha}')
    print(f'beta = {beta}')
    for df in rolling_dfs:
        temp_weighted_similarity = []
        for i in range(len(df)):
            temp_weighted_similarity.append((1/(1+np.array(df.iloc[i].cumprod_diffs))*alpha + (beta * np.array(df.iloc[i].quantdare))))

        df['weighted_similarity'] = temp_weighted_similarity

    weighted_rolling = sort_by_metric(rolling_dfs, 'weighted_similarity')
    #top_vs_bottom_histogram(weighted_rolling,k=20, plots=True)
add_to_error_dict(weighted_rolling, 'weighted_similarity', aggregation=agg, seed=False)

ADDING ProposedAdjusted
alpha = 0.5
beta = 0.5


100%|██████████| 172/172 [05:44<00:00,  2.00s/it]


In [6]:
#-- Add the remaining baselines
print("ADDING ProposedPearson")
for alpha in [0.5]:
    print('='*20)
    beta=1-alpha
    print(f'alpha = {alpha}')
    print(f'beta = {beta}')
    for df in rolling_dfs:
        temp_weighted_similarity = []
        for i in range(len(df)):
            temp_weighted_similarity.append((1/(1+np.array(df.iloc[i].cumprod_diffs))*alpha + (beta * np.array(df.iloc[i].correlations))))

        df['equal_weight_similarity'] = temp_weighted_similarity

    equal_rolling = sort_by_metric(rolling_dfs, 'equal_weight_similarity')
add_to_error_dict(equal_rolling, 'equal_weight_similarity', aggregation=agg, seed=False)
del equal_rolling

print("ADDING Shape")
shape_rolling = sort_by_metric(rolling_dfs, 'shape_measure')
add_to_error_dict(shape_rolling, 'shape_measure', aggregation=agg, seed=False)
del shape_rolling

print("ADDING PearsonOnly")
correlation_rolling = sort_by_metric(rolling_dfs, 'correlations')
add_to_error_dict(correlation_rolling, 'correlations', aggregation=agg, seed=False)
del correlation_rolling

print("ADDING CumulativeOnly")
cumprod_rolling = sort_by_metric(rolling_dfs, 'cumprod_diffs', reverse=False)
add_to_error_dict(cumprod_rolling, 'cumprod_diffs', aggregation=agg, seed=False)
del cumprod_rolling

print("ADDING AdjustedOnly")
qd_rolling = sort_by_metric(rolling_dfs, 'quantdare')
add_to_error_dict(qd_rolling, 'quantdare', aggregation=agg, seed=False)
del qd_rolling
print('QD Done')

ADDING ProposedPearson
alpha = 0.5
beta = 0.5


  0%|          | 0/172 [00:00<?, ?it/s]

ADDING Shape


  0%|          | 0/172 [00:00<?, ?it/s]

ADDING PearsonOnly


...

ADDING CumulativeOnly


  0%|          | 0/172 [00:00<?, ?it/s]

ADDING AdjustedOnly


100%|██████████| 172/172 [01:22<00:00,  2.09it/s]


QD Done


In [7]:
#-- Display table 1
df = pd.DataFrame(np.array(list(error_dict['weighted_similarity'].values()))[:,0], index=[1,2,5,10,25,50], columns=['weighted_similarity'])
for metric in list(error_dict.keys())[1:]:
    #print('='*20)
    #print(metric)
    df[metric]=np.array(list(error_dict[metric].values()))[:,0]
display(df.T)

<ipython-input-7-0150f9dfa66b>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  df = pd.DataFrame(np.array(list(error_dict['weighted_similarity'].values()))[:,0], index=[1,2,5,10,25,50], columns=['weighted_similarity'])
<ipython-input-7-0150f9dfa66b>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  df[metric]=np.array(list(error_dict[metric].values()))[:,0]


,1,2,5,10,25,50
weighted_similarity,0.0887294,0.088549,0.0882301,0.0882748,0.0882213,0.0882663
equal_weight_similarity,0.0893349,0.0890898,0.0884871,0.0883485,0.0882824,0.0884208
shape_measure,0.0887873,0.0887696,0.0888952,0.0888617,0.0890942,0.0892939
correlations,0.0909763,0.091096,0.0909417,0.0909025,0.0908742,0.090789
cumprod_diffs,0.0910655,0.0911922,0.0907582,0.0906796,0.0904136,0.090249
quantdare,0.0910664,0.0909644,0.0907465,0.090755,0.0906666,0.0905435


In [10]:

#-- Create a DF with all the underlying values which make up the aggregate errors table
#- This will allow us to perform a Tukey HSD test
raw_dict = {}
for metric in error_dict.keys():
    for k in [10]:
        key = metric+'_'+str(k)
        raw_dict[key] = error_dict[metric][k][2]
temp_df = pd.DataFrame(raw_dict)

temp_df.rename(columns={"weighted_similarity_"+str(k): "proposed_"+str(k),
                   "equal_weight_similarity_"+str(k): "pearson_cumprod_"+str(k),
                  "shape_measure_"+str(k): "shape_"+str(k),
                  "correlations_"+str(k): "pearson_"+str(k),
                  "cumprod_diffs_"+str(k): "cumprod_diffs_"+str(k),
                  "quantdare_"+str(k): "adjusted_corr_"+str(k)},
              inplace=True)

#-- Save to csv and reload
#- Without this step the Tukey test gives an error
temp_df.to_csv('error_df_10.csv')
temp_df = pd.read_csv('error_df_10.csv', index_col=0)

temp_df.head()

,proposed_10,pearson_cumprod_10,shape_10,pearson_10,cumprod_diffs_10,adjusted_corr_10
0,0.031954,0.095433,0.067251,0.057734,0.020649,0.057734
1,0.056500,0.031954,0.071173,0.095433,0.060591,0.294980
2,0.042043,0.056500,0.230242,0.021645,0.002037,0.019239
3,0.030988,0.057734,0.000214,0.041380,0.035972,0.056500
4,0.035923,0.042043,0.056500,0.056056,0.089966,0.031954


In [11]:
#-- Perform a Tukey HSD Test
#- Note that you need to do the save to CSV and reload in step for this to work
x_vals = []
y_vals = []
for column in temp_df.columns:
    y_vals.append(temp_df[column].values)
    x_vals.append([column for _ in range(len(temp_df[column].values))])

x_vals=np.array(x_vals).flatten()
y_vals=np.array(y_vals).flatten()

# DataFrame.
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi
mcDate = multi.MultiComparison(y_vals,x_vals)
Results = mcDate.tukeyhsd()
#print(Results)

temp_df = pd.DataFrame(data=Results._results_table.data[1:], columns=Results._results_table.data[0])
temp_df[(temp_df.group1.str.contains('proposed')) | (temp_df.group2.str.contains('proposed'))]

,group1,group2,meandiff,p-adj,lower,upper,reject
3,adjusted_corr_10,proposed_10,-0.0025,0.0010,-0.0031,-0.0018,True
7,cumprod_diffs_10,proposed_10,-0.0024,0.0010,-0.0031,-0.0018,True
10,pearson_10,proposed_10,-0.0026,0.0010,-0.0033,-0.0020,True
12,pearson_cumprod_10,proposed_10,-0.0001,0.9000,-0.0007,0.0006,False
14,proposed_10,shape_10,0.0006,0.0993,-0.0001,0.0012,False


### Second Evaluation - Trading Sim

In [51]:

def get_returns(rolling_dfs,k, aggregation=False, seed=False):
    if seed!=False:
        np.random.seed(seed)
        tickers = list(rolling_dfs[0].ticker.value_counts().index)
        tickers = np.random.choice(tickers, int(len(tickers)*0.2), replace=False)
        
        rolling_dfs = [df[~df.ticker.isin(tickers)] for df in rolling_dfs]
        
        x_top = np.array([abs((np.array([np.array(xi)[:k].mean() for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values]))) for df in rolling_dfs])
        
    return x_top


n_years = 172/12
def get_cumulative_performance(rolling_dfs, k, n, subsample=True):
    #-- Get returns of top-k most similar for the 80% of assests for each RW
    
    if subsample:
        performance_list = []
        for _ in tqdm(range(100)):
            #-- Get the next month returns of top-k most similar cases
            tickers = list(rolling_dfs[0].ticker.value_counts().index)
            tickers = np.random.choice(tickers, int(len(tickers)*0.2), replace=False)

            temp_rolling_dfs = [df[~df.ticker.isin(tickers)] for df in rolling_dfs]
            #temp_rolling_dfs = [df for df in rolling_dfs]

            #x_top = np.array([(np.array([np.array(xi)[:k].mean() for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values])) for df in temp_rolling_dfs])
            #-- Line below for random
            x_top = np.array([(np.array([np.random.choice(np.array(xi),k,replace=False).mean() for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values])) for df in temp_rolling_dfs])
            
            temp = np.array([df[df.start_date==df.start_date.iloc[0]].ticker for df in temp_rolling_dfs])
            #print(x_top.shape)
            #print(temp.shape)
            #temp = get_returns(weighted_rolling, k=k, seed=seed)


            monthly_returns = []
            for i in range(172):
                #-- Get the index of the top-n highest average top-k sim returns
                #print(len(x_top[i,:]))
                idxs = x_top[i,:].argsort()[-n:][::-1]
                #-- Store the return of the month of trading
                #print(temp_rolling_dfs[i].ticker.values[idxs])
                #print(i, idxs)
                #print(temp_rolling_dfs[i].next_month.values[idxs])
                #print(idxs)
                monthly_returns.append(temp_rolling_dfs[i].next_month.values[idxs].mean())

            #performance_list.append(np.cumprod(1+np.array(monthly_returns))[-1]**(1/n_years)-1)
            performance_list.append(monthly_returns)
        return performance_list
    
    else:
        for _ in range(1):
            #-- Get the next month returns of top-k most similar cases
            #tickers = list(rolling_dfs[0].ticker.value_counts().index)
            #tickers = np.random.choice(tickers, int(len(tickers)*0.2), replace=False)

            #temp_rolling_dfs = [df[~df.ticker.isin(tickers)] for df in rolling_dfs]
            temp_rolling_dfs = [df for df in rolling_dfs]

            x_top = np.array([(np.array([np.array(xi)[:k].mean() for xi in df[df.start_date==df.start_date.iloc[0]].sim_next_month.values])) for df in temp_rolling_dfs])
            #temp = np.array([df[df.start_date==df.start_date.iloc[0]].ticker for df in temp_rolling_dfs])
            #print(x_top.shape)
            #print(temp.shape)
            #temp = get_returns(weighted_rolling, k=k, seed=seed)


            monthly_returns = []
            for i in range(172):
                #-- Get the index of the top-n highest average top-k sim returns
                #print(len(x_top[i,:]))
                idxs = x_top[i,:].argsort()[-n:][::-1]
                #-- Store the return of the month of trading
                #print(temp_rolling_dfs[i].ticker.values[idxs])
                #print(i, idxs)
                #print(temp_rolling_dfs[i].next_month.values[idxs])
                #print(idxs)
                monthly_returns.append(temp_rolling_dfs[i].next_month.values[idxs].mean())

        return monthly_returns


In [52]:
k=10
n=5

"""
Run for each metric with 80% subsampling for evaluation
"""

proposed_performance = get_cumulative_performance(weighted_rolling, k=k, n=n)
proposed_plotting = get_cumulative_performance(weighted_rolling, k=k, n=n, subsample=False)
temp=np.cumprod(1+np.array(proposed_performance), axis=1)**(1/n_years)-1
print(np.mean(temp[:,-1]))

for alpha in [0.5]:
    print('='*20)
    beta=1-alpha
    print(f'alpha = {alpha}')
    print(f'beta = {beta}')
    for df in rolling_dfs:
        temp_weighted_similarity = []
        for i in range(len(df)):
            temp_weighted_similarity.append((1/(1+np.array(df.iloc[i].cumprod_diffs))*alpha + (beta * np.array(df.iloc[i].correlations))))

        df['equal_weight_similarity'] = temp_weighted_similarity

    equal_rolling = sort_by_metric(rolling_dfs, 'equal_weight_similarity')
pearson_cumprod_performance = get_cumulative_performance(equal_rolling, k=k, n=n)
#print(np.mean(pearson_cumprod_performance))
temp=np.cumprod(1+np.array(pearson_cumprod_performance), axis=1)**(1/n_years)-1
print(np.mean(temp[:,-1]))
pearson_cumprod_plotting = get_cumulative_performance(equal_rolling, k=k, n=n, subsample=False)
del equal_rolling
print('Pearson-Cumprod Done')


shape_rolling = sort_by_metric(rolling_dfs, 'shape_measure')
shape_performance = get_cumulative_performance(shape_rolling, k=k, n=n)
temp=np.cumprod(1+np.array(shape_performance), axis=1)**(1/n_years)-1
print(np.mean(temp[:,-1]))
shape_plotting = get_cumulative_performance(shape_rolling, k=k, n=n, subsample=False)
del shape_rolling
print('Shape Done')

correlation_rolling = sort_by_metric(rolling_dfs, 'correlations')
correlation_performance = get_cumulative_performance(correlation_rolling, k=k, n=n)
temp=np.cumprod(1+np.array(correlation_performance), axis=1)**(1/n_years)-1
print(np.mean(temp[:,-1]))
correlation_plotting = get_cumulative_performance(correlation_rolling, k=k, n=n, subsample=False)
del correlation_rolling
print('Correlation Done')

cumprod_rolling = sort_by_metric(rolling_dfs, 'cumprod_diffs', reverse=False)
cumprod_performance = get_cumulative_performance(cumprod_rolling, k=k, n=n)
temp=np.cumprod(1+np.array(cumprod_performance), axis=1)**(1/n_years)-1
print(np.mean(temp[:,-1]))
cumprod_plotting = get_cumulative_performance(cumprod_rolling, k=k, n=n, subsample=False)
del cumprod_rolling
print('Cumprod Done')

qd_rolling = sort_by_metric(rolling_dfs, 'quantdare')
qd_performance = get_cumulative_performance(qd_rolling, k=k, n=n)
temp=np.cumprod(1+np.array(qd_performance), axis=1)**(1/n_years)-1
print(np.mean(temp[:,-1]))
qd_plotting = get_cumulative_performance(qd_rolling, k=k, n=n, subsample=False)
del qd_rolling
print('QD Done')


100%|██████████| 100/100 [06:32<00:00,  3.92s/it]


0.14479005222250627
alpha = 0.5
beta = 0.5


100%|██████████| 100/100 [07:03<00:00,  4.24s/it]


0.14214693273367934


  0%|          | 0/172 [00:00<?, ?it/s]

...

0.14397886572301938


  0%|          | 0/172 [00:00<?, ?it/s]

Cumprod Done


100%|██████████| 100/100 [07:14<00:00,  4.35s/it]


0.1493771283157008
QD Done


In [71]:
def I(x):
    if x<0:
        return x
    else:
        return 0
    
def LSV(temp):
    #-- lower semi variance funciton
    summ=0
    for val in temp:
        summ+=(I(val-temp.mean()))**2
    return (summ/len(temp))**0.5



    
all_100_means= {}
for vals,name in zip([proposed_performance, pearson_cumprod_performance, shape_performance, cumprod_performance, qd_performance, correlation_performance], ['proposed', 'pearson_cumprod', 'shape', 'cumprod', 'adjusted',
       'pearson']):
    all_100_means[name]=np.array(vals)
    
    
n_years = (all_100_means['proposed'].shape[1])/12
for metric in ['proposed', 'pearson_cumprod', 'shape', 'cumprod', 'adjusted','pearson']:
    print('='*20)
    print(metric)
    print('='*20)
    #rand_mus = (1 + (np.cumprod(1+np.array(rand_mean),axis=0)-1)[-1,:])**(1/n_years) - 1
    if metric=='adjusted':
        mus = (1+np.array(all_100_means[metric])).cumprod(axis=1)[:,-1]**(1/n_years) - 1
    if metric=='shape':
        mus = (1+np.array(all_100_means[metric])).cumprod(axis=1)[:,-1]**(1/n_years) - 1
    else:
        mus = (1+np.array(all_100_means[metric])).cumprod(axis=1)[:,-1]**(1/n_years) - 1
    sigmas = np.std(np.array(all_100_means[metric]), axis=1)*(12**0.5)
    lsvs = np.array([LSV(np.array(all_100_means[metric])[i,:]) for i in range(np.array(all_100_means[metric]).shape[0])])*(12**0.5)

    mu = np.mean(mus)
    sigma = np.mean(sigmas)
    lsv = np.mean(lsvs)
    print(metric,' Annualised Return',round(mu*100,2))
    print(metric,' St Dev of  Return',round(np.std(mus)*100,2))
    print(metric,' Annualised STD',round(sigma*100,2))
    print(metric,' Annualised LSV',round(lsv*100,2))
    print(metric,' Sharpe',round((mu-0.01)/sigma,2))
    print(metric,' Sortino',round((mu-0.01)/lsv,2))
    

proposed
proposed  Annualised Return 15.85
proposed  St Dev of  Return 3.55
proposed  Annualised STD 19.2
proposed  Annualised LSV 13.92
proposed  Sharpe 0.77
proposed  Sortino 1.07
pearson_cumprod
pearson_cumprod  Annualised Return 14.21
pearson_cumprod  St Dev of  Return 3.24
pearson_cumprod  Annualised STD 18.86
pearson_cumprod  Annualised LSV 13.83
pearson_cumprod  Sharpe 0.7
pearson_cumprod  Sortino 0.96
shape
shape  Annualised Return 15.17
shape  St Dev of  Return 3.15
shape  Annualised STD 19.14
shape  Annualised LSV 13.95
shape  Sharpe 0.74
shape  Sortino 1.02
cumprod
cumprod  Annualised Return 14.4
cumprod  St Dev of  Return 3.65
cumprod  Annualised STD 19.09
cumprod  Annualised LSV 13.97
cumprod  Sharpe 0.7
cumprod  Sortino 0.96
adjusted
adjusted  Annualised Return 15.35
adjusted  St Dev of  Return 3.27
adjusted  Annualised STD 19.02
adjusted  Annualised LSV 13.89
adjusted  Sharpe 0.75
adjusted  Sortino 1.03
pearson
pearson  Annualised Return 14.71
pearson  St Dev of  Return 

#### Tukey Test

In [73]:
temp_df = pd.DataFrame({'proposed':(1+np.array(proposed_performance)).cumprod(axis=1)[:,-1]**(1/n_years) - 1,
                        'pearson_cumprod':(1+np.array(pearson_cumprod_performance)).cumprod(axis=1)[:,-1]**(1/n_years) - 1,
                        'shape':(1+np.array(shape_performance)).cumprod(axis=1)[:,-1]**(1/n_years) - 1,
                        'cumprod':(1+np.array(cumprod_performance)).cumprod(axis=1)[:,-1]**(1/n_years) - 1,
                        'adjusted':(1+np.array(qd_performance)).cumprod(axis=1)[:,-1]**(1/n_years) - 1,
                        'pearson':(1+np.array(correlation_performance)).cumprod(axis=1)[:,-1]**(1/n_years) - 1
                       })
#temp_df.to_csv('one_hundred_80pc_performance.csv', index=False)

In [75]:
"""
Tukey Test on Trading

"""
#-- Load the dataframe of underlying returns
temp_df=pd.read_csv('one_hundred_80pc_performance.csv')
temp_df=1000*(1+temp_df)**(172/12)
print("Final Accumulated Value for Each Metric")
display(temp_df.mean())

#-- Arrange the data for a tukey test
x_vals = []
y_vals = []
for column in temp_df.columns:
    y_vals.append(temp_df[column].values)
    x_vals.append([column for _ in range(len(temp_df[column].values))])
    #print('KS Test - ',column)
    #print(round(stats.ks_2samp(temp_df['proposed_'+str(k)], temp_df[column], alternative='less')[1],4))

x_vals=np.array(x_vals).flatten()
y_vals=np.array(y_vals).flatten()
y_vals = y_vals.astype(float)

#-- Perform Tukey test
mcDate = multi.MultiComparison(y_vals,x_vals)
Results = mcDate.tukeyhsd()
#print(Results)

#-- Store results
temp_df = pd.DataFrame(data=Results._results_table.data[1:], columns=Results._results_table.data[0])
#-- Extract tukey just for the proposed metric
temp_df[(temp_df.group1.str.contains('proposed')) | (temp_df.group2.str.contains('proposed'))]


#temp_df

Final Accumulated Value for Each Metric


proposed           8564.544752
pearson_cumprod    6755.428505
shape              8058.941613
cumprod            6971.994215
adjusted           8112.214224
pearson            6356.014217
dtype: float64

,group1,group2,meandiff,p-adj,lower,upper,reject
3,adjusted,proposed,452.3305,0.6327,-399.8040,1304.4651,False
7,cumprod,proposed,1592.5505,0.0010,740.4160,2444.6851,True
10,pearson,proposed,2208.5305,0.0010,1356.3960,3060.6651,True
12,pearson_cumprod,proposed,1809.1162,0.0010,956.9817,2661.2508,True
14,proposed,shape,-505.6031,0.5297,-1357.7377,346.5314,False
